<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/Dacon/blob/main/DeepChem_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install deepchem
!pip install rdkit
!pip install scikit-learn


In [ ]:
# !pip install keras==2.4.3
# !pip install tensorflow==2.3.0
# !pip uninstall -y keras tensorflow
# !pip install tensorflow
# !pip install tf_keras
!pip install tensorflow==2.10.0
!pip install deepchem==2.6.0



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 879.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 39.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  

ERROR: Ignored the following versions that require a different python version: 2.6.0.dev20220112162333 Requires-Python >=3.7,<3.10; 2.6.0.dev20220114040838 Requires-Python >=3.7,<3.10; 2.6.0.dev20220118010103 Requires-Python >=3.7,<3.10; 2.6.0.dev20220118135955 Requires-Python >=3.7,<3.10; 2.6.1 Requires-Python >=3.7,<3.10; 2.6.1.dev20220118213325 Requires-Python >=3.7,<3.10; 2.6.1.dev20220121040525 Requires-Python >=3.7,<3.10; 2.6.1.dev20220121191633 Requires-Python >=3.7,<3.10; 2.6.1.dev20220127040719 Requires-Python >=3.7,<3.10; 2.6.1.dev20220127201742 Requires-Python >=3.7,<3.10; 2.6.1.dev20220208165000 Requires-Python >=3.7,<3.10; 2.6.1.dev20220210194849 Requires-Python >=3.7,<3.10; 2.6.1.dev20220211203047 Requires-Python >=3.7,<3.10; 2.6.1.dev20220218060535 Requires-Python >=3.7,<3.10; 2.6.1.dev20220218194027 Requires-Python >=3.7,<3.10; 2.6.1.dev20220219162715 Requires-Python >=3.7,<3.10; 2.6.1.dev20220222165950 Requires-Python >=3.7,<3.10; 2.6.1.dev20220225175824 Requires-Pytho

In [ ]:
import deepchem as dc
import pandas as pd
from sklearn.model_selection import train_test_split
from deepchem.models import GraphConvModel
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'


# 1. 데이터 로드 및 전처리
chembl_data = pd.read_csv('/content/drive/MyDrive/데이콘/train.csv')

# SMILES 데이터와 pIC50 값을 DeepChem의 Dataset으로 변환
featurizer = dc.feat.ConvMolFeaturizer()
X = featurizer.featurize(chembl_data['Smiles'].values)
y = chembl_data['pIC50'].values

# DeepChem 데이터셋으로 변환
dataset = dc.data.NumpyDataset(X, y)

# 2. 학습 및 검증 데이터 분리
train_dataset, valid_dataset, test_dataset = dc.splits.RandomSplitter().train_valid_test_split(dataset, frac_train=0.7, frac_valid=0.15, frac_test=0.15, seed=42)

# 3. GraphConvModel 모델 정의 및 학습
graphconv_model = GraphConvModel(n_tasks=1, mode='regression', dropout=0.2, model_dir="graphconv_model", batch_normalize=False) # Set batch_normalize to False
graphconv_model.fit(train_dataset, nb_epoch=50)

# 4. 검증 데이터 평가
train_scores = graphconv_model.evaluate(train_dataset, [dc.metrics.Metric(dc.metrics.mean_squared_error)])
valid_scores = graphconv_model.evaluate(valid_dataset, [dc.metrics.Metric(dc.metrics.mean_squared_error)])

print("GraphConvModel Train MSE:", train_scores['mean_squared_error'])
print("GraphConvModel Valid MSE:", valid_scores['mean_squared_error'])

# 5. 테스트 데이터 예측 및 결과 저장
test_data = pd.read_csv('/content/drive/MyDrive/데이콘/test.csv')
test_X = featurizer.featurize(test_data['Smiles'].values)
test_dataset = dc.data.NumpyDataset(test_X)

test_preds_graphconv = graphconv_model.predict(test_dataset)

# 결과 저장
submit = pd.read_csv('/content/drive/MyDrive/데이콘/sample_submission.csv')
submit['pIC50'] = test_preds_graphconv
submit.to_csv('graphconv_submit.csv', index=False)



/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(448,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(448, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(3102,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_16:0", shape=(3102, 64), dtype=float32), dense

GraphConvModel Train MSE: 0.5014609786739228
GraphConvModel Valid MSE: 0.6915797035922572
